In [157]:
import netCDF4
from netCDF4 import Dataset, num2date, date2num, date2index
import pandas as pd
import numpy as np
import datetime
import os

In [158]:
#directory = "/Users/minhyuk/Dropbox/RA/RA_COVID19/*github/weather-conditions-COVID19/WeatherDataCollection/templates"
directory = "/Users/minhyuk/Dropbox/RA/RA_COVID19/pollutant/nc"

os.chdir(directory)
#os.getcwd()
#os.system("dir")

## Set Template and Files

Data for gases (ozone, sulfur dioxide, and nitrogen dioxide) and data for particulate matter are stored in a different batch of files. Further, different templates are used depending on the dataset to be gathered. Change following boolean to decide which files to read.

In [159]:
read_pm_files = True #surface level data
key = 0 #date proxy variables while vary from 1 to XXXX (4 obs per day)

In [160]:
template_file_name = "US_pollution_template.xlsx"
#template_file_name = "global_simple_pollution_template.xlsx"
#template_file_name = "city granular_template.xlsx"
#template_file_name = "1072_pollution_template.xlsx"

In [161]:
template_file_name[:-14]

'US_pollution'

In [162]:
if read_pm_files:
    no_df = pd.read_excel(template_file_name)
    no2_df = pd.read_excel(template_file_name)
    so2_df = pd.read_excel(template_file_name)
    co_df = pd.read_excel(template_file_name)
    pm2_df = pd.read_excel(template_file_name)
    pm10_df = pd.read_excel(template_file_name)
else:
    no2_df = pd.read_excel(template_file_name)
    o3_df = pd.read_excel(template_file_name)
    so2_df = pd.read_excel(template_file_name)

In [163]:
if read_pm_files:
#    lookup_lats, lookup_longs = pm_df["Lat"], pm_df["Long"]
    lookup_lats, lookup_longs = no2_df["Lat"], no2_df["Long"]
else:
    lookup_lats, lookup_longs = no2_df["Lat"], no2_df["Long"]

The following returns the lat/long indices closest to a desired point, and is used for reading from the netCDF4 layer.

In [164]:
def getClosest_ij(lats, longs, latpt, longpt):
    #Manhattan Distance
    i = abs(lats - latpt).argmin()
    j = abs(longs - longpt).argmin()
    return i,j

In [165]:
def fill_dataframes(month, month_length, year):
    for i in range(month_length*4):
        no_col = []
        no2_col = []
        so2_col = []
        co_col = []
        pm2_col = []
        pm10_col = []
        
        for lat, long in zip(lookup_lats, lookup_longs):
            iy, ix = getClosest_ij(lats, lons, lat, long)
            if read_pm_files:
                no_col.append(no[i][iy][ix])
                no2_col.append(no2[i][iy][ix])
                so2_col.append(so2[i][iy][ix])
                co_col.append(co[i][iy][ix])
                pm2_col.append(pm2[i][iy][ix])
                pm10_col.append(pm10[i][iy][ix])
            else:
                so2_col.append(so2[i][iy][ix])
                no2_col.append(no2[i][iy][ix])
                o3_col.append(o3[i][iy][ix])
        
        global key
        key = key + 1 #observation at 2019-01-01 3am is 1
        #print(key)
    
        if read_pm_files:
            no_df[key] = no_col
            no2_df[key] = no2_col
            so2_df[key] = so2_col
            co_df[key] = co_col
            pm2_df[key] = pm2_col
            pm10_df[key] = pm10_col
        else:
            so2_df[key] = so2_col
            no2_df[key] = no2_col
            o3_df[key] = o3_col

The following are maps from file names to month, number of days, and year triples, with the triples being used to group the data by data. Notice that for the gases argument map (ozone, sulfur dioxide, nitrogen dioxide), July 2017 is commented out - this month is handled separately due to a shift in ECMWF's reporting structure.

In [166]:
pm_argument_map = {
    "1Jan2019":(1,31,2019),
    "2Feb2019":(2,28,2019),
    "3March2019":(3,31,2019),
    "4April2019":(4,30,2019),
    "5May2019":(5,31,2019),
    "6June2019":(6,30,2019),
    "7July2019":(7,31,2019),
    "8Auguest2019":(8,31,2019),
    "9September2019":(9,30,2019),
    "10October2019":(10,31,2019),
    "11November2019":(11,30,2019),
    "Dec2019":(12,31,2019),
    "Jan2020":(1,31,2020),
    "Feb2020":(2,29,2020),
    "March2020":(3,31,2020),
    "April2020":(4,30,2020)
}

In [167]:
gases_argument_map = {
    "1 Jan2019":(1,31,2019),
    "2 Feb2019":(2,28,2019),
    "3 March2019":(3,31,2019),
    "4 April2019":(4,30,2019),
    "5 May2019":(5,31,2019),
    "6 June2019":(6,30,2019),
    #"7 July2019":(7,31,2019),
    "8 Aug2019":(8,31,2019),
    "9 Sep2019":(9,30,2019),
    "10 Oct2019":(10,31,2019),
    "11 Nov2019":(11,30,2019),
    "12 Dec2019":(12,31,2019),
    "Jan2020":(1,31,2020),
    "Feb2020":(2,29,2020),
    "March2020":(3,31,2020),
    "April2020":(4,30,2020)
}

In [168]:
new_argument_map1 = {
    "2019-01":(1,31,2019),
    "2019-02":(2,28,2019),
    "2019-03":(3,31,2019),
    "2019-04":(4,30,2019),
    "2019-05":(5,31,2019),
    "2019-06":(6,30,2019)
}
new_argument_map2 = {
    "2019-07":(7,31,2019),
    "2019-08":(8,31,2019),
    "2019-09":(9,30,2019),
    "2019-10":(10,31,2019),
    "2019-11":(11,30,2019),
    "2019-12":(12,31,2019)
}
new_argument_map3 = {
    "2020-01":(1,31,2020),
    "2020-02":(2,29,2020),
    "2020-03":(3,31,2020),
    "2020-04":(4,30,2020),    
    "2020-05":(5,31,2020),
    "2020-06":(6,30,2020),
    "2020-07":(7,31,2020),
    "2020-08":(8,31,2020)
}

In [169]:
if read_pm_files:
#    month_argument_map = pm_argument_map
    month_argument_map = new_argument_map3

else:
#    month_argument_map = gases_argument_map
    month_argument_map = new_argument_map3
    
for file in month_argument_map:
    f = Dataset(file + ".nc", format="NETCDF4")

    lats, lons = f.variables["latitude"][:], f.variables["longitude"][:]
    if read_pm_files:
        ch4 = f.variables['tc_ch4'][:]
        no = f.variables['tc_no'][:]
        no2 = f.variables['tcno2'][:]
        co = f.variables['tcco'][:]
        so2 = f.variables['tcso2'][:]
        pm2 = f.variables['pm2p5'][:]
        pm10 = f.variables['pm10'][:]
#        pm = f.variables['pm2p5'][:]
    else:
        no2 = f.variables['no2'][:]
        o3 = f.variables['go3'][:]
        so2 = f.variables['so2'][:]

    arg = month_argument_map[file]
    fill_dataframes(arg[0], arg[1], arg[2])
    print("{0}-{1} data fetching done".format(arg[2], arg[0]))

2020-1 data fetching done
2020-2 data fetching done
2020-3 data fetching done
2020-4 data fetching done
2020-5 data fetching done
2020-6 data fetching done
2020-7 data fetching done
2020-8 data fetching done


In [170]:
if read_pm_files:
#    doc_name = template_file_name[:-14]+"_pm"
    doc_name = template_file_name[:-14]+"_new-data3"
else:
    doc_name = template_file_name[:-14]+"_gases"

In [171]:
writer1 = pd.ExcelWriter(doc_name+'_no.xlsx', engine='xlsxwriter')
writer2 = pd.ExcelWriter(doc_name+'_no2.xlsx', engine='xlsxwriter')
writer3 = pd.ExcelWriter(doc_name+'_so2.xlsx', engine='xlsxwriter')
writer4 = pd.ExcelWriter(doc_name+'_co.xlsx', engine='xlsxwriter')
writer5 = pd.ExcelWriter(doc_name+'_pm2p5.xlsx', engine='xlsxwriter')
writer6 = pd.ExcelWriter(doc_name+'_pm10.xlsx', engine='xlsxwriter')

In [172]:
if read_pm_files: 
    no_df.to_excel(writer1, sheet_name = "no", index = False)
    print("no to excel is completed")
    no2_df.to_excel(writer2, sheet_name = "no2", index = False)
    print("no2 to excel is completed")
    so2_df.to_excel(writer3, sheet_name = "so2", index = False)
    print("so2 to excel is completed")
    co_df.to_excel(writer4, sheet_name = "co", index = False)
    print("co to excel is completed")
    pm2_df.to_excel(writer5, sheet_name = "pm2p5", index = False)
    print("pm2.5 to excel is completed")
    pm10_df.to_excel(writer6, sheet_name = "pm10", index = False)
    print("pm10 to excel is completed")

else:
    o3_df.to_excel(writer, sheet_name = "o3", index = False)
    no2_df.to_excel(writer, sheet_name = "no2", index = False)
    so2_df.to_excel(writer, sheet_name = "so2", index = False)

no to excel is completed
no2 to excel is completed
so2 to excel is completed
co to excel is completed
pm2.5 to excel is completed
pm10 to excel is completed


In [173]:
writer1.close()
writer2.close()
writer3.close()
writer4.close()
writer5.close()
writer6.close()

## Manually Reading July2019 Gas Data

ECMWF shifted their reporting for ozone, sulfur dioxide, and nitrogen dioxide from 60 levels of vertical resolution to 137 (see https://confluence.ecmwf.int/display/COPSRV/Implementation+of+IFS+cycle+46r1), so we read both and patch them together at their transition date, which was July 9 2019.

In [ ]:
data60 = Dataset("7 July2019 level 60 up to here.nc", format="NETCDF4")

In [ ]:
data137 = Dataset("7 Jul2019 level 137 from here.nc", format="NETCDF4")

In [ ]:
template_file_name = "US_pollution_template.xlsx"
#template_file_name = "global_simple_pollution_template.xlsx"
#template_file_name = "city granular_template.xlsx"
#template_file_name = "1072_pollution_template.xlsx"

In [ ]:
no2_df = pd.read_excel(template_file_name)
o3_df = pd.read_excel(template_file_name)
so2_df = pd.read_excel(template_file_name)

In [ ]:
lookup_lats, lookup_longs = no2_df["Lat"], no2_df["Long"]

In [ ]:
lats, lons = data60.variables["latitude"][:], data60.variables["longitude"][:]
no2 = data60.variables['no2'][:]
o3 = data60.variables['go3'][:]
so2 = data60.variables['so2'][:]

fill_dataframes(7, 31, 2019)

In [ ]:
writer = pd.ExcelWriter(template_file_name[:-14] + ' July - Level 60.xlsx', engine='xlsxwriter')

o3_df.to_excel(writer, sheet_name = "o3", index = False)
no2_df.to_excel(writer, sheet_name = "no2", index = False)
so2_df.to_excel(writer, sheet_name = "so2", index = False)

writer.close()

### Level 137 Data

In [ ]:
no2_df = pd.read_excel(template_file_name)
o3_df = pd.read_excel(template_file_name)
so2_df = pd.read_excel(template_file_name)

In [ ]:
lookup_lats, lookup_longs = no2_df["Lat"], no2_df["Long"]

In [ ]:
lats, lons = data137.variables["latitude"][:], data137.variables["longitude"][:]
no2 = data137.variables['no2'][:]
o3 = data137.variables['go3'][:]
so2 = data137.variables['so2'][:]

fill_dataframes(7, 23, 2019) #only reading 23 days since data begins on July 9.

In [ ]:
writer = pd.ExcelWriter(template_file_name[:-14] + ' July - Level 137.xlsx', engine='xlsxwriter')

o3_df.to_excel(writer, sheet_name = "o3", index = False)
no2_df.to_excel(writer, sheet_name = "no2", index = False)
so2_df.to_excel(writer, sheet_name = "so2", index = False)

writer.close()